In [5]:
from tqdm import tqdm
import warnings
import matplotlib.animation as animation 
from IPython import display 

warnings.filterwarnings('ignore')

In [298]:
""" Smoke Plume
Hot smoke is emitted from a circular region at the bottom.
The simulation computes the resulting air flow in a closed box.
"""
#from phi.flow import *  # minimal dependencies
# from phi.torch.flow import *
# from phi.tf.flow import *
from phi.jax.flow import *

Res = 200   # Resolution

Walls = Obstacle(union(
    [Box(x=(20, 60), y=(50, 51)), #bottom wall
    Box(x=(80, 100), y=(70, 71)), #top bar
    Box(x=(20, 21), y=(50, 65)),
    Box(x=(45, 46), y=(40, 50))
    ]
))

velocity = StaggeredGrid(0, x=Res, y=Res, bounds=Box(x=100, y=100))  # or CenteredGrid(...)
smoke = CenteredGrid(0, ZERO_GRADIENT, x=2*Res, y=2*Res, bounds=Box(x=100, y=100))
inflow = 0.2 * resample(Sphere(x=60, y=10, radius=5), to=smoke, soft=True)
pressure = None


@jit_compile  # Only for PyTorch, TensorFlow and Jax
def step(v, s, p, dt=1.):
    s = advect.mac_cormack(s, v, dt) + inflow
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, Walls, Solve('auto', 1e-5, 0, x0=p))
    return v, s, p

In [299]:
for _ in range(1):
    velocity, smoke, pressure = step(velocity, smoke, pressure)

In [300]:
trj = iterate(step, batch(time=300), velocity, smoke, pressure)

In [301]:
plot(vis.overlay(trj[1],Walls.geometry), alpha=1, show_color_bar=False, animate='time')

In [ ]:
# plot(trj[1], show_color_bar=False, animate='time')